<a href="https://colab.research.google.com/github/kd303/TableDetection/blob/master/table_detection_pycoco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code to connect to google drive, this is to ensure collab lists does not lose the data

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
%cd /content/drive/My\ Drive/

In [0]:
# %mkdir table_detection

%cd table_detection

%pwd

In [0]:
%pwd

In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

Following ***pip install*** has to happen everytime Collab is restarted

In [0]:
!apt-get install -qq protobuf-compiler python-tk

In [0]:
!pip install tensorflow-gpu==1.14

In [0]:
!pip install -q pycocotools 

Following are for reference these does not need to happen everytime for Collab. However remember the working directory is always inside `my drive/table_detection`

In [0]:
#@title
%cd /content/drive/My Drive/table_detection/models/research

In [0]:
#@title
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
#@title
%ls -ltr object_detection/protos/*.*

All one time commands till now are done.

In [0]:
#@title
from google.colab import files
uploaded = files.upload()

In [0]:
%pwd
%ls -ltr /content/drive/My\ Drive/table_detection/pre_trained_model/faster_rcnn_inception_v2_coco_2018_01_28

In [0]:
%cp table_detection_pipeline\ \(1\).config table_detection_pipeline.config

In [0]:
%cp ./models/research/object_detection/* ./pipeline_config/

In [0]:
%ls -ltr ./pipeline_config/

%cp -r ./models/research/object_detection/* ./pipeline_config/

In [0]:
%rm -r pipeline_config  

In [0]:
import os
import shutil
import glob
import requests
import tarfile

MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/drive/My Drive/table_detection/trained_models/'

req = requests.get(DOWNLOAD_BASE + MODEL_FILE)
with open(DEST_DIR + MODEL_FILE, 'wb' ) as f:
  f.write(req.content) 



In [0]:
tar = tarfile.open(DEST_DIR + MODEL_FILE)
tar.extractall(path=DEST_DIR)
tar.close()

In [0]:
%pwd
# %cd /content/drive/My\ Drive/table_detection/models/research
%set_env PYTHONPATH=/project/pylib/src:/env/python:/content/drive/My Drive/table_detection/models/research/slim

In [0]:
! echo $PYTHONPATH

In [0]:
import sys
sys.path.append('/content/drive/My Drive/table_detection/models/research/slim')

In [0]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0

In [0]:
%tensorflow_version

In [0]:
%tensorflow_version 1.x

In [0]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml

In [0]:
%pwd

In [0]:
!pip install tensorflow-object-detection-api

In [0]:
!pip install --upgrade tf_slim

In [0]:
!pip3 install numpy==1.17.4

In [0]:
import numpy as np
np.__version__

In [0]:
  
import os
os.environ['PYTHONPATH'] += ":/content/drive/My Drive/table_detection/models/research:/content/drive/My Drive/table_detection/models/research/slim"

! echo $PYTHONPATH

In [0]:
%cd /content/drive/My\ Drive/table_detection/models/research/object_detection/

In [0]:
%rm -r /content/drive/My\ Drive/table_detection/trained_models/*

In [0]:
%cp /content/drive/My Drive/table_detection/pre_trained_model/faster_rcnn_inception_v2_coco_2018_01_28/table_detection_pipeline\ (1).config /content/drive/My Drive/table_detection/pre_trained_model/faster_rcnn_inception_v2_coco_2018_01_28/table_detection_pipeline.config

In [0]:
%load_ext tensorboard

In [0]:
!python model_main.py \
    --pipeline_config_path=/content/drive/My\ Drive/table_detection/pre_trained_model/faster_rcnn_inception_v2_coco_2018_01_28/table_detection_pipeline.config \
    --model_dir=/content/drive/My\ Drive/table_detection/trained_models \
    --alsologtostderr \
    --num_train_steps=50000 \
    --num_eval_steps=500

In [0]:
# Tensorboard 

%load_ext tensorboard

%tensorboard --logdir=/content/drive/My\ Drive/table_detection/trained_models

In [0]:
%pwd

In [0]:
%cd /content/drive/My Drive/table_detection

In [0]:
lst = os.listdir('trained_models')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

In [0]:
print(last_model)

In [0]:
!python ./models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/drive/My\ Drive/table_detection/pre_trained_model/faster_rcnn_inception_v2_coco_2018_01_28/table_detection_pipeline.config \
    --output_directory=/content/drive/My\ Drive/table_detection/fine_tuned_model \
    --trained_checkpoint_prefix=/content/drive/My\ Drive/table_detection/trained_models/$last_model

In [0]:
%cd /content/drive/My Drive/table_detection/data

from google.colab import files
from os import path

uploaded = files.upload()
  


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("./")
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
%matplotlib inline




from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


In [0]:
# function to convert image to np array
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [0]:
%tensorflow_version 1.x

In [0]:
# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/drive/My Drive/table_detection/fine_tuned_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/drive/My Drive/table_detection/pre_trained_model/faster_rcnn_inception_v2_coco_2018_01_28', 'labelmap.pbtxt')

NUM_CLASSES = 1
# label to text

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
import glob
PATH_TO_TEST_IMAGES_DIR = '/content/drive/My Drive/table_detection/data/test_images/'
TEST_IMAGES = os.listdir(PATH_TO_TEST_IMAGES_DIR)

print(TEST_IMAGES)
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, img) for img in TEST_IMAGES]
print(TEST_IMAGE_PATHS)

In [0]:
 # If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.

# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 2) ]

# Size, in inches, of the output images. # will have to see why this is imp
IMAGE_SIZE = (100, 80)




def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:
IMAGE_SIZE = (100, 80) 

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  print('Printing images '+image_path)
  # print(output_dict['detection_boxes'])
  # print(output_dict['detection_classes'])
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

In [0]:
TEST_IMAGE_PATHS